# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

In [3]:
data2 = pd.DataFrame(data = {"Price in 1 Year" : [140, 110, 80],
                            "Probability" : [0.25, 0.50, 0.25]})
data2

Price in 1 Year  Probability
0              140         0.25
1              110         0.50
2               80         0.25

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [4]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    data['Possible Return'] = ((data['Price in 1 Year']/P) -1)
    exp_ret = (data['Possible Return']*data['Probability']).sum()
    return exp_ret

print(exp_return(data, 40)) # should output 0.07500000000000002
print(exp_return(data2, 100)) # should output 0.1

0.07500000000000002
0.10000000000000003


Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [5]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    data['Possible Return'] = ((data['Price in 1 Year']/P) -1)
    exp_ret = exp_return(data,P)
    vol = 0
    for index, row in data.iterrows():
        vol += row['Probability'] * pow((row['Possible Return']-exp_ret), 2) 
    vol = pow(vol, 0.5)
    return vol

print(volatility(data,40)) # should output 0.21301408404140795
print(volatility(data2,100)) # should output 0.212 

0.21301408404140795
0.21213203435596423


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [6]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    avg_return = exp_return(data, P)
    sd_ir = volatility(data, P)
    n = len(data.index)
    z = 1.96
    sd_i = z*sd_ir/pow(n,0.5)
    return [avg_return-sd_i,avg_return+sd_i]

print(interval(data, 40)) # should output [-0.16604812797447732, 0.3160481279744774]

[-0.16604812797447732, 0.3160481279744774]


Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [7]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return rf + beta*(market_return - rf)

print(exp_return_capm(-0.6,0.1, 0.02)) # should output -0.028

-0.028


### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [8]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)
    df = df[pd.to_numeric(df['RET'], errors='coerce').notnull()]
    df['RET'] = df['RET'].astype(float) # applying transformation converts from obj to float
    df = df.groupby(['CUSIP']).filter(lambda x: x.RET.size >=500)
    df = df.reset_index(drop=True)
    return df
tmp = clean_data('monthlycrsp_mod6.csv')

display(tmp)

PERMNO        date     CUSIP       RET
0        10065  1970-01-30  00621210 -0.144375
1        10065  1970-02-27  00621210  0.105882
2        10065  1970-03-31  00621210  0.017857
3        10065  1970-04-30  00621210 -0.078947
4        10065  1970-05-29  00621210 -0.020952
...        ...         ...       ...       ...
403311   84129  2022-08-31  98970110  0.016315
403312   84129  2022-09-30  98970110 -0.075777
403313   84129  2022-10-31  98970110  0.021235
403314   84129  2022-11-30  98970110  0.005583
403315   84129  2022-12-30  98970110 -0.051332

[403316 rows x 4 columns]

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [9]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    group = data.groupby('CUSIP').RET.mean()
    output = pd.DataFrame(data = {
        "CUSIP" : group.keys(),
        "Average Return" : group 
    })
    output = output.reset_index(drop=True)
    
    return output
avg_return(tmp)


CUSIP  Average Return
0    00036110        0.012326
1    00095710        0.011339
2    00105510        0.018398
3    00108M10        0.011101
4    00120410        0.012135
..        ...             ...
674  G6359F13        0.019803
675  G7665A10        0.013261
676  G8994E10        0.013812
677  H2717810        0.011593
678  N5946510        0.019437

[679 rows x 2 columns]

In [10]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    group = data.groupby('CUSIP').RET.std()
    output = pd.DataFrame(data = {
        "CUSIP" : group.keys(),
        "Volatility" : group 
    })
    output = output.reset_index(drop=True)

    return output
security_volatilities(tmp)

CUSIP  Volatility
0    00036110    0.122475
1    00095710    0.089881
2    00105510    0.099828
3    00108M10    0.171732
4    00120410    0.051050
..        ...         ...
674  G6359F13    0.205250
675  G7665A10    0.129473
676  G8994E10    0.090320
677  H2717810    0.131243
678  N5946510    0.134147

[679 rows x 2 columns]

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [11]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    return ret_data['Average Return'].corr(vol_data['Volatility'])
correlation(avg_return(tmp), security_volatilities(tmp))

0.4483709072705961